## ロジスティック回帰 (2)

- 別のデータでの例

In [ ]:
import pandas as pd
import seaborn as sns

# Irisデータセットの読み込み
iris = sns.load_dataset('iris')

# 2値データへの分類のため、species から setosa を除外
# - versicolor, virginica の2種だけにした df を作成
df = iris.query('species!="setosa"')

# versicolor, virginica だけになっていることを確認
# - 0, 50 行目を表示
df.iloc[[0, 50]]

In [ ]:
# 散布図でデータの重なりを見る
sns.pairplot(df, hue='species')

上記の散布図で versicolor, virginica の重なり具合を見ると：

- sepal_width は、かなりの重なりがある
- petal_width は、多少の重なりがある

In [ ]:
# speciesを 0, 1 にするためにダミー変数化
df = pd.get_dummies(data=df, drop_first=True)

# 確認のため 0, 50 行目を表示
df.iloc[[0, 50]]

<hr>

### かなりの重なりがある sepal_width を説明変数としてロジスティック回帰モデルを作成する

In [ ]:
from sklearn.linear_model import LogisticRegression

X = df[['sepal_width']]
Y = df.species_virginica

# ロジスティック回帰モデルを作成
model = LogisticRegression(C=10000.0)

# 学習
model.fit(X, Y)

# 評価（正確度: Accuracy）
model.score(X, Y)

In [ ]:
# 散布図にシグモイド曲線を重ねて表示する
import matplotlib.pyplot as plt
import numpy as np

# 標準シグモイド関数の定義
def sigmoid(x, b0, b1):
    return 1/(1 + np.exp(-(b0 + b1 * x)))

# 散布図
sns.scatterplot(x=X.iloc[:,0], y=Y)

# 学習したモデルからパラメータの取得
b0 = model.intercept_[0]
b1 = model.coef_[0][0]

# y=0.5となる分割線
_x = -b0 / b1
plt.plot([_x, _x], [0, 1], color='lightblue')
print('Boundary:', _x)

# シグモイド曲線
_max = X.iloc[:,0].max()
_min = X.iloc[:,0].min()
_step = (_max - _min) / 100
_X = np.arange(_min, _max + _step, _step)
plt.plot(_X, sigmoid(_X, b0, b1), color='brown')

In [ ]:
# 教師データについてモデルからの予測値を計算
Y_predict = model.predict(X)
# 混合行列
# - 1行目1列：実際が0、予測が0 (真陰性：TN)
# - 1行目2列：実際が0、予測が1 (偽陽性：FP)
# - 2行目1列：実際が1、予測が0 (偽陰性：FN)
# - 2行目2列：実際が1、予測が1 (真陽性：TP)
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(Y, Y_predict)
matrix

In [ ]:
# 正確度、適合度、再現率（感度）、特異度
from sklearn.metrics import accuracy_score, precision_score, recall_score
# 特異度 = TN / (FP + TN)
specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0])

print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
    accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
    recall_score(Y, Y_predict), specificity))